# AGN Spectral Decomposition and Black‐Hole Mass Estimation

This notebook demonstrates how to download a Sloan Digital Sky Survey (SDSS) spectrum for a selected active galactic nucleus (AGN), decompose the spectrum into host galaxy and AGN components using principal component analysis (PCA) with galaxy and quasar eigen‐spectra, fit the Hβ emission line, and compute physical quantities such as the black‑hole mass, bolometric luminosity, accretion rate and e‑folding growth time.

**Note:** This notebook is designed for execution in a cloud notebook environment such as Google Colab with internet access and requires the `astroquery`, `lmfit`, and `matplotlib` packages. The eigen‐spectra files (e.g., `eigen_galaxies.npy` and `eigen_quasars.npy`) must be provided separately.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.sdss import SDSS
from astropy.table import Table

# File path to the DR16 quasar table provided in this repository
csv_path = '/content/table_dr16_qso_krc213.csv'  # adjust path if necessary
df = pd.read_csv(csv_path)
print(f'The table contains {len(df)} AGN entries.')
df.head()


### Select a target AGN

Choose an AGN from the catalog by index, `specObjID`, or coordinates.  The SDSS spectral identification is given by `plate`, `mjd` and `fiberID`, which uniquely identify a spectrum in SDSS.  You can also use the right ascension and declination to query the nearest spectrum.  In this example we select the first row of the catalog:

In [ ]:
# Select the first AGN in the list
idx = 0  # change this index to analyze a different object
row = df.iloc[idx]

plate = int(row['plate'])
mjd = int(row['mjd'])
fiber = int(row['fiberID'])
ra = row['ra']
dec = row['dec']

print(f'Selected AGN index {idx}: plate={plate}, mjd={mjd}, fiberID={fiber}, ra={ra:.5f}, dec={dec:.5f}')


### Download the SDSS spectrum

Use `astroquery.sdss.SDSS` to retrieve the calibrated spectrum.  You can query by `plate`, `mjd` and `fiberID` or by sky coordinates within a search radius:

In [ ]:
# Query the SDSS archive for the spectrum
# Using plate, mjd, fiberID:
spec_list = SDSS.get_spectra(plate=plate, mjd=mjd, fiberID=fiber)

# Alternatively: use sky coordinates if plate/mjd/fiber are not known
# coords = SkyCoord(ra* u.deg, dec* u.deg, frame='icrs')
# spec_list = SDSS.get_spectra(matches=coords, radius=2*u.arcsec)

# Retrieve the first spectrum and convert to a Pandas DataFrame
spec = spec_list[0]
data = spec[1].data  # table with wavelength and flux
wave = data['loglam']  # log10 of wavelength in angstroms
wavelength = 10**wave
flux = data['flux']

# Plot the full spectrum
plt.figure(figsize=(10,4))
plt.plot(wavelength, flux)
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux (10^-17 erg s^-1 cm^-2 Å^-1)')
plt.title('SDSS Spectrum')
plt.show()


### Perform PCA decomposition

Load precomputed galaxy and quasar eigen‐spectra (principal components) and perform a least‐squares fit to reconstruct the host galaxy and AGN components.  The eigenspectra must be aligned to the same wavelength grid as the observed spectrum (e.g., via interpolation).  The decomposition uses separate sets of galaxy and quasar eigenspectra to efficiently and reliably separate the AGN and host components【429195297408580†L24-L34】.  For demonstration we assume the eigenspectra are stored as NumPy arrays with shape `(n_components, n_pixels)`.


In [ ]:
# Load the eigen‐spectra arrays (replace with actual file paths)
# eigen_gal = np.load('/content/eigen_galaxies.npy')  # shape: (Ng, n_pixels)
# eigen_quasar = np.load('/content/eigen_quasars.npy')  # shape: (Nq, n_pixels)

# Interpolate eigenspectra onto the observed wavelength grid
# (this step is required if the wavelength grids differ)
# from scipy.interpolate import interp1d

# For example:
# interp_gal = [interp1d(wave_eig, eig, kind='linear', bounds_error=False, fill_value=0)(wavelength) for eig in eigen_gal]
# interp_qso = [interp1d(wave_eig, eig, kind='linear', bounds_error=False, fill_value=0)(wavelength) for eig in eigen_quasar]

# Stack the galaxy and quasar eigenspectra into a design matrix
# A = np.vstack([interp_gal, interp_qso]).T  # shape: (n_pixels, Ng+Nq)
# b = flux  # observed flux vector

# Solve the linear least squares problem to obtain coefficients
# coeffs, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
# gal_coeffs = coeffs[:len(interp_gal)]
# qso_coeffs = coeffs[len(interp_gal):]

# Reconstruct the galaxy and AGN spectra
# galaxy_spec = np.dot(gal_coeffs, interp_gal)
# agn_spec = np.dot(qso_coeffs, interp_qso)

# Plot decomposition
# plt.figure(figsize=(10,4))
# plt.plot(wavelength, flux, label='Observed', color='black', lw=1)
# plt.plot(wavelength, galaxy_spec, label='Host galaxy', color='tab:blue')
# plt.plot(wavelength, agn_spec, label='AGN component', color='tab:red', alpha=0.7)
# plt.legend()
# plt.xlim(3500, 7000)
# plt.xlabel('Wavelength (Å)')
# plt.ylabel('Flux')
# plt.title('Eigenspectrum decomposition')
# plt.show()


### Subtract the galaxy component

Subtract the reconstructed host galaxy spectrum from the observed spectrum to produce a pure AGN spectrum.  This step removes starlight and other host contributions, isolating the broad emission lines.  In code:


In [ ]:
# pure_agn = flux - galaxy_spec
# plt.figure(figsize=(10,4))
# plt.plot(wavelength, pure_agn)
# plt.xlabel('Wavelength (Å)')
# plt.ylabel('Flux')
# plt.title('AGN spectrum after galaxy subtraction')
# plt.show()


### Fit the Hβ emission line

Estimate the continuum around the Hβ emission line and model the line profile as a sum of a narrow and a broad Gaussian component plus continuum.  We fit a linear continuum to wavelength windows free from strong emission (e.g., 4750–4830 and 4920–5000 Å in the rest frame) and subtract it.  Then we use `scipy.optimize.curve_fit` or `lmfit` to fit Gaussian functions to the continuum‐subtracted line region.

The full width at half maximum (FWHM) of the broad Hβ component can be converted to velocity using the relation FWHM_velocity = \(c 	imes \Delta\lambda / \lambda_0\), where \(c\) is the speed of light and \(\Delta\lambda\) is the measured FWHM in angstroms【982153414888892†L136-L144】.  The same relation can be used to convert line widths into km s⁻¹ when estimating black‐hole masses.


In [ ]:
from scipy.optimize import curve_fit

# Define Gaussian model for broad and narrow components

def gaussian(x, amp, center, sigma):
    return amp * np.exp(-0.5 * ((x - center)/sigma)**2)

# Define a combined model: continuum + narrow + broad

def hb_model(x, cont_slope, cont_intercept, amp_n, cen_n, sigma_n, amp_b, cen_b, sigma_b):
    cont = cont_slope * x + cont_intercept
    narrow = gaussian(x, amp_n, cen_n, sigma_n)
    broad = gaussian(x, amp_b, cen_b, sigma_b)
    return cont + narrow + broad

# Convert rest wavelength 4861 Å to observed frame
z = 0.0  # placeholder redshift; replace with measured redshift for the AGN
lambda_hb_rest = 4861.0
lambda_obs = lambda_hb_rest * (1 + z)

# Select wavelength region around Hβ (±200 Å)
# mask = (wavelength > lambda_obs - 200) & (wavelength < lambda_obs + 200)
x_data = wavelength  # [mask]
y_data = flux       # [mask]

# Initial guesses: slope ~0, intercept ~median flux, approximate amplitudes and sigmas
p0 = [0.0, np.median(y_data), 
      np.max(y_data), lambda_obs, 5.0,  # narrow component
      np.max(y_data)/2, lambda_obs, 20.0]  # broad component

# Fit the model to the data
# popt, pcov = curve_fit(hb_model, x_data, y_data, p0=p0)

# Extract fitted parameters
# cont_slope, cont_intercept, amp_n, cen_n, sigma_n, amp_b, cen_b, sigma_b = popt

# Compute FWHM of broad component in angstroms and convert to km/s
# fwhm_angstrom = 2 * np.sqrt(2*np.log(2)) * sigma_b
# fwhm_kms = (fwhm_angstrom / lambda_hb_rest) * 3e5

# Plot fit
# model_fit = hb_model(x_data, *popt)
# plt.figure(figsize=(8,4))
# plt.plot(x_data, y_data, label='Spectrum')
# plt.plot(x_data, model_fit, label='Model fit')
# plt.xlabel('Wavelength (Å)')
# plt.ylabel('Flux')
# plt.legend()
# plt.title('Hβ line fit')
# plt.show()



### Compute monochromatic luminosity and bolometric luminosity

After subtracting the continuum and fitting the broad Hβ component, compute the monochromatic continuum luminosity at 5100 Å (rest frame) using the relation \(L_{\lambda} = 4\pi d_{L}^2 f_{\lambda}\), where \(d_{L}\) is the luminosity distance and \(f_{\lambda}\) is the flux density at 5100 Å.  Astronomers often report \(\lambda L_{\lambda}(5100\,	ext{Å})\).  The bolometric luminosity can be estimated from the continuum luminosity with a bolometric correction factor \(L_{	ext{bol}} pprox 9 	imes \lambda L_{\lambda}(5100\,	ext{Å})\)【924124004071945†L150-L159】.


In [ ]:
# Example calculation for monochromatic luminosity at 5100 Å
from astropy.cosmology import Planck18 as cosmo

# Suppose we measured flux density at observed 5100*(1+z) Å after continuum fit:
# f_lambda = cont_intercept + cont_slope * (5100*(1+z))
# d_L = cosmo.luminosity_distance(z).to(u.cm).value
# lambda_rest = 5100.0
# lam_L_lam = (lambda_rest * 1e-8) * 4 * np.pi * d_L**2 * f_lambda  # erg/s
# L_bol = 9 * lam_L_lam

# print('lambda L_lambda (5100 Å) =', lam_L_lam, 'erg/s')
# print('Bolometric luminosity =', L_bol, 'erg/s')


### Estimate black‐hole mass

Use a single‐epoch virial estimator based on the broad Hβ line width and continuum luminosity.  Following Vestergaard & Peterson (2006) (as used in the *easyspec* code), the black‐hole mass can be computed as

\[\log M_{ullet} = 6.91 + 2 \log\left(rac{\mathrm{FWHM}(\mathrm{H}eta)}{1000\;\mathrm{km\;s}^{-1}}ight) + 0.5 \log\left(rac{\lambda L_{\lambda}(5100\,	ext{Å})}{10^{44}\;\mathrm{erg\;s}^{-1}}ight)\]【505461062941029†L818-L844】.

Compute the Eddington luminosity as \(L_{	ext{Edd}} = 1.26 	imes 10^{38} (M_{ullet}/M_{\odot})\,\mathrm{erg\;s}^{-1}\) for ionized gas【191268095570323†L220-L233】 and obtain the Eddington ratio \(L_{	ext{bol}} / L_{	ext{Edd}}\).


In [ ]:
# M_sun = 1.989e33  # g
# FWHM_kms = fwhm_kms
# lam_L_lam_44 = lam_L_lam / 1e44
# M_bh = 10**(6.91 + 2*np.log10(FWHM_kms/1000) + 0.5*np.log10(lam_L_lam_44))  # solar masses

# Compute Eddington luminosity and Eddington ratio
# L_edd = 1.26e38 * M_bh
# edd_ratio = L_bol / L_edd

# print('Black hole mass =', M_bh, 'M_sun')
# print('Eddington ratio =', edd_ratio)


### Accretion rate and exponential growth

Assuming a radiative efficiency \(\etapprox 0.1\), the mass accretion rate is \(\dot M = L_{	ext{bol}} / (\eta c^2)\)【244003009144014†L33-L38】.  The Salpeter (e‐folding) time for black hole growth is

\[ t_{	ext{S}} pprox 4.5	imes10^{7}\ rac{\eta}{0.1}\ \left(rac{L_{	ext{bol}}}{L_{	ext{Edd}}}ight)^{-1}\ 	ext{yr} \]【893725374850037†L33-L56】.

The time required for the black hole to grow from its current mass \(M_{ullet}\) to a final mass \(M_{	ext{final}}\) (e.g., \(10^{10} M_{\odot}\)) under continuous Eddington‐ratio accretion is

\[ t = t_{	ext{S}}\,\ln\left(rac{M_{	ext{final}}}{M_{ullet}}ight). \]

Compute the accretion rate, the Salpeter time, and the growth time for the AGN.


In [ ]:
# eta = 0.1
# c_cm_s = 2.99792458e10
# Mdot = L_bol / (eta * c_cm_s**2) / M_sun  # in M_sun/s
# Mdot_per_year = Mdot * (3600*24*365)

# # Salpeter time in years
# t_salpeter = 4.5e7 * (eta / 0.1) * (L_bol / L_edd)**(-1)

# # Target mass (e.g., 1e10 M_sun)
# M_final = 1e10
# t_growth = t_salpeter * np.log(M_final / M_bh)

# print('Mass accretion rate =', Mdot_per_year, 'M_sun/yr')
# print('Salpeter time =', t_salpeter/1e6, 'Myr')
# print('Growth time to 1e10 M_sun =', t_growth/1e9, 'Gyr')



## Summary

This notebook outlines a workflow for downloading SDSS spectra, isolating the AGN component via PCA, fitting the Hβ emission line, and deriving physical quantities such as black‐hole mass, bolometric luminosity, accretion rate and growth time.  The approach follows recent studies that use galaxy and AGN templates to isolate the AGN contribution in SDSS spectra【563814819262489†L96-L103】 and adopt virial black‐hole mass estimators from the literature【505461062941029†L818-L844】.  The conversion from spectral line width to velocity uses the standard relation between FWHM and velocity【982153414888892†L136-L144】.  Bolometric luminosities are estimated using a 5100 Å continuum bolometric correction【924124004071945†L150-L159】, and Eddington luminosities follow the classical formula for hydrogen‐dominated gas【191268095570323†L220-L233】.  Accretion rates and e‑folding times are computed following standard accretion physics【244003009144014†L33-L38】【893725374850037†L33-L56】.

Use this notebook as a template to explore a sample of AGN.  Replace placeholder values with measurements from your own spectra and adjust the fit regions and initial parameters as necessary.
